In [5]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

In [11]:
import segmentation_models_3D as sm
model1 = sm.Unet('resnet34', encoder_weights='imagenet')

Segmentation Models: using `tf.keras` framework.


ValueError: Invalid axis: 4

In [1]:
from classification_models_3D.tfkeras import Classifiers
ResNet18, preprocess_input = Classifiers.get('resnet18')
model = ResNet18(input_shape=(128, 128, 128, 3), weights='imagenet')

2022-03-07 15:22:26.649425: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-03-07 15:22:26.649447: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-07 15:22:27.462818: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-03-07 15:22:27.482384: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-07 15:22:27.482406: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: cauliflower
2022-03-07 15:22:27.482412: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: cauliflower
2022-03-07 15:22:27.482467: I tensorflow/stream_exec

In [3]:
patient_id = 'Brats18_TCIA09_620_1'
contrast = 't1ce'
tumour = 'LGG'
filename = f'/home/atom/Documents/datasets/brats/MICCAI_BraTS_2018_Data_Training/{tumour}/{patient_id}/{patient_id}_{contrast}.nii.gz'

In [4]:
nii_img  = nib.load(filename)
nii_data = nii_img.get_fdata()

In [13]:
nii_data.shape

(240, 240, 155)

In [14]:
model = sm.Unet(backbone_name='resnet50', input_shape=(240, 240, 155, 1), classes=3, activation='softmax')

185450496/185445896 [==============================] - 22s 0us/step


ValueError: axes don't match array